In [2]:
# A warning message from DGL will appear which seems to be related to an open issue in the DGL library, 
# it won't hurt the execution of the program, please ignore it.
import quartz

Using backend: pytorch
[06:20:30] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/zikunli/anaconda3/envs/quantum/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.10.2.so: cannot open shared object file: No such file or directory


In [3]:
quartz_context = quartz.QuartzContext(gate_set=['h', 'cx', 't', 'tdg'], filename='../bfs_verified_simplified.json')
parser = quartz.PyQASMParser(context=quartz_context)
my_dag = parser.load_qasm(filename="barenco_tof_3_opt_path/subst_history_39.qasm")
init_graph = quartz.PyGraph(context=quartz_context, dag=my_dag)

In [4]:
from dgl import save_graphs, load_graphs
save_graphs('from_graphs.dat', [init_graph.to_dgl_graph()])
g = load_graphs('from_graphs.dat')
# g[0].ndata
g[0][0]

Graph(num_nodes=58, num_edges=154,
      ndata_schemes={'gate_type': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'reversed': Scheme(shape=(), dtype=torch.int64), 'src_idx': Scheme(shape=(), dtype=torch.int64), 'dst_idx': Scheme(shape=(), dtype=torch.int64)})

In [5]:
# available_xfer_matrix = init_graph.get_available_xfers_matrix(context=quartz_context)
# all_nodes = init_graph.all_nodes()
# for node in all_nodes:
#     print(init_graph.available_xfers(context=quartz_context, node=node))

In [6]:
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit

def check(graph):
    graph.to_qasm(filename='best.qasm')
    qc_origin = QuantumCircuit.from_qasm_file('barenco_tof_3_opt_path/subst_history_39.qasm')
    qc_optimized = QuantumCircuit.from_qasm_file('best.qasm')
    return Statevector.from_instruction(qc_origin).equiv(Statevector.from_instruction(qc_optimized))

In [7]:
# Optimizing with BFS
import heapq
from concurrent.futures import ProcessPoolExecutor
import copy

candidate_hq = []
heapq.heappush(candidate_hq, init_graph)
hash_set = set()
hash_set.add(init_graph.hash())
best_graph = init_graph
best_gate_cnt = init_graph.gate_count
max_gate_cnt = 64

budget = 5_000_000

while candidate_hq != [] and budget >= 0:
    first_candidate = heapq.heappop(candidate_hq)
    all_nodes = first_candidate.all_nodes()
    
    def ax(i):
        node = all_nodes[i]
        return first_candidate.available_xfers(context=quartz_context, node=node)
    
    with ProcessPoolExecutor(max_workers=64) as executor:
        results = executor.map(ax, list(range(len(all_nodes))), chunksize=2)
        appliable_xfers_nodes = []
        for r in results:
            appliable_xfers_nodes.append(r)
        
    for i in range(len(all_nodes)):
        node = all_nodes[i]
        appliable_xfers = appliable_xfers_nodes[i]
        for xfer in appliable_xfers:
            new_graph = first_candidate.apply_xfer(xfer=quartz_context.get_xfer_from_id(id=xfer), node=node)
            new_hash = new_graph.hash()
            if new_hash not in hash_set:
                hash_set.add(new_hash)
                new_cnt = new_graph.gate_count
                if new_cnt <= max_gate_cnt:
                    heapq.heappush(candidate_hq, new_graph)
                if new_cnt < best_gate_cnt:
                    best_graph = new_graph
                    best_gate_cnt = new_cnt
                budget -= 1
                if budget % 10_000 == 0:
                    print(f'{budget}: minimum gate count is {best_gate_cnt}, correctness: {check(best_graph)}')


4990000: minimum gate count is 52, correctness: True
4980000: minimum gate count is 52, correctness: True
4970000: minimum gate count is 52, correctness: True
4960000: minimum gate count is 46, correctness: True
4950000: minimum gate count is 46, correctness: True
4940000: minimum gate count is 46, correctness: True
4930000: minimum gate count is 46, correctness: True
4920000: minimum gate count is 46, correctness: True
4910000: minimum gate count is 46, correctness: True
4900000: minimum gate count is 46, correctness: True
4890000: minimum gate count is 46, correctness: True
4880000: minimum gate count is 46, correctness: True
4870000: minimum gate count is 46, correctness: True
4860000: minimum gate count is 46, correctness: True
4850000: minimum gate count is 46, correctness: True
4840000: minimum gate count is 46, correctness: True
4830000: minimum gate count is 46, correctness: True
4820000: minimum gate count is 46, correctness: True
4810000: minimum gate count is 46, correctness